In [18]:
# multi-class classification with Keras
import time  
from sklearn.metrics import accuracy_score
import numpy as np  
import matplotlib.pyplot as plt
import os

import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from sklearn.model_selection import train_test_split
import warnings
from keras.models import Model
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
import keras as K
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_pickle('./Discriptor_hfo2_del.pkl')
df

,structure,name
0,"[[1.38867986 2.3864784 4.36482979] Hf, [3.587...",Ag_HfO2_cat_3.125_222_m.cif
1,"[[1.40181386 2.3944349 4.3885007 ] Hf, [3.606...",Ag_HfO2_cat_3.125_222_o.cif
2,"[[-8.57055872 -7.98390112 -6.2853237 ] Hf, [-6...",Ag_HfO2_cat_3.125_222_p-o.cif
3,"[[-8.73105937 2.58337834 3.5977214 ] Hf, [-6...",Ag_HfO2_cat_3.125_222_t.cif
4,"[[1.37820795 2.39009891 4.37039843] Hf, [3.622...",Ag_HfO2_cat_6.25_122_m.cif
...,...,...
1720,"[[ 1.26076304 -5.11876687 1.26075093] Hf, [ 1...",Zr_HfO2_inter_6.25_212_t.cif
1721,"[[ 1.2262443 -10.12402918 -3.43749088] Hf, ...",Zr_HfO2_inter_6.25_221_m.cif
1722,"[[1.39997305 0.20795941 1.85144277] Hf, [1.451...",Zr_HfO2_inter_6.25_221_o.cif
1723,"[[-8.97250742 -0.20890362 1.50328354] Hf, [-8...",Zr_HfO2_inter_6.25_221_p-o.cif


In [7]:
from matminer.featurizers.structure import XRDPowderPattern
xrd = XRDPowderPattern()
df = xrd.fit_featurize_dataframe(df, 'structure')
df

,structure,name,xrd_0,xrd_1,xrd_2,xrd_3,xrd_4,xrd_5,xrd_6,xrd_7,...,xrd_118,xrd_119,xrd_120,xrd_121,xrd_122,xrd_123,xrd_124,xrd_125,xrd_126,xrd_127
0,"[[1.38867986 2.3864784 4.36482979] Hf, [3.587...",Ag_HfO2_cat_3.125_222_m.cif,6.274031e-39,4.868723e-35,2.251073e-31,6.201119e-28,1.017786e-24,9.952886e-22,5.798926e-19,2.013038e-16,...,0.001413,0.001787,0.002558,0.003072,0.003398,0.004110,0.004837,0.004574,0.003350,0.001935
1,"[[1.40181386 2.3944349 4.3885007 ] Hf, [3.606...",Ag_HfO2_cat_3.125_222_o.cif,4.804097e-37,2.421562e-33,7.436718e-30,1.391453e-26,1.586199e-23,1.101662e-20,4.661662e-18,1.201808e-15,...,0.001368,0.001436,0.002387,0.004332,0.006577,0.008093,0.008030,0.006211,0.003627,0.001582
2,"[[-8.57055872 -7.98390112 -6.2853237 ] Hf, [-6...",Ag_HfO2_cat_3.125_222_p-o.cif,6.457440e-39,4.488355e-35,1.872707e-31,4.690381e-28,7.051837e-25,6.364323e-22,3.447926e-19,1.121294e-16,...,0.008355,0.007088,0.005549,0.005928,0.007303,0.007191,0.005535,0.004443,0.004490,0.004156
3,"[[-8.73105937 2.58337834 3.5977214 ] Hf, [-6...",Ag_HfO2_cat_3.125_222_t.cif,2.636581e-118,7.576613e-111,1.230048e-103,1.128227e-96,5.846746e-90,1.711966e-83,2.832447e-77,2.648142e-71,...,0.005475,0.008558,0.008262,0.006027,0.005012,0.004838,0.004702,0.004740,0.004365,0.003063
4,"[[1.37820795 2.39009891 4.37039843] Hf, [3.622...",Ag_HfO2_cat_6.25_122_m.cif,1.018675e-12,6.942950e-11,2.802394e-09,6.698709e-08,9.482654e-07,7.949604e-06,3.946734e-05,1.160398e-04,...,0.002221,0.002619,0.002921,0.002846,0.002909,0.003670,0.004553,0.004376,0.003123,0.001688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,"[[ 1.26076304 -5.11876687 1.26075093] Hf, [ 1...",Zr_HfO2_inter_6.25_212_t.cif,8.754284e-117,2.117514e-109,2.908113e-102,2.267648e-95,1.003966e-88,2.523724e-82,3.602001e-76,2.918940e-70,...,0.012107,0.013373,0.009723,0.004710,0.001975,0.002136,0.003536,0.004352,0.004388,0.003838
1721,"[[ 1.2262443 -10.12402918 -3.43749088] Hf, ...",Zr_HfO2_inter_6.25_221_m.cif,2.100243e-40,2.473450e-36,1.688501e-32,6.681341e-29,1.532464e-25,2.037425e-22,1.570135e-19,7.013858e-17,...,0.001659,0.002858,0.004195,0.004837,0.004773,0.004220,0.003423,0.002731,0.002120,0.001409
1722,"[[1.39997305 0.20795941 1.85144277] Hf, [1.451...",Zr_HfO2_inter_6.25_221_o.cif,2.145256e-13,1.668879e-11,7.677879e-10,2.088952e-08,3.361138e-07,3.198269e-06,1.799759e-05,5.989417e-05,...,0.002767,0.003910,0.004815,0.005397,0.005761,0.005669,0.004737,0.003112,0.001529,0.000540
1723,"[[-8.97250742 -0.20890362 1.50328354] Hf, [-8...",Zr_HfO2_inter_6.25_221_p-o.cif,1.155537e-42,2.052994e-38,2.056702e-34,1.161809e-30,3.700653e-27,6.646632e-24,6.731394e-21,3.844048e-18,...,0.004711,0.004409,0.004602,0.004804,0.004691,0.004902,0.005736,0.006533,0.006282,0.004394


In [9]:
labels_true = pd.read_pickle('./data_labels/labels_hfo2.pkl')
y_true = labels_true['labels_0_4']
# df.insert(loc=2, column='y_true', value=y_true)
y_true.values

array(['m', 'o', 'p-o', ..., 'o', 'unknown', 't'], dtype=object)

In [11]:
from sklearn.utils import shuffle
X = df.iloc[:,2:]
X, y_true = shuffle(X, y_true, random_state=0)
X

,xrd_0,xrd_1,xrd_2,xrd_3,xrd_4,xrd_5,xrd_6,xrd_7,xrd_8,xrd_9,...,xrd_118,xrd_119,xrd_120,xrd_121,xrd_122,xrd_123,xrd_124,xrd_125,xrd_126,xrd_127
1613,8.654797e-37,4.295206e-33,1.296280e-29,2.379037e-26,2.655163e-23,1.802059e-20,7.437627e-18,1.866756e-15,2.849235e-13,2.644580e-11,...,0.001504,0.001881,0.003371,0.005566,0.007572,0.008305,0.006949,0.004196,0.001803,0.000564
124,1.598668e-13,2.575932e-11,2.193994e-09,9.878105e-08,2.351043e-06,2.958071e-05,1.967613e-04,6.921024e-04,1.290261e-03,1.300372e-03,...,0.001231,0.001450,0.002087,0.002689,0.002643,0.002234,0.002171,0.002495,0.002227,0.001202
648,1.658078e-113,2.238268e-106,1.755022e-99,7.993117e-93,2.114527e-86,3.249178e-80,2.899994e-74,1.503433e-68,4.527247e-63,7.918586e-58,...,0.004550,0.007633,0.009787,0.009882,0.007337,0.004388,0.004208,0.005946,0.005953,0.003535
249,3.498679e-12,2.449046e-10,1.026943e-08,2.579600e-07,3.881647e-06,3.498978e-05,1.889533e-04,6.115429e-04,1.189226e-03,1.412274e-03,...,0.007133,0.006231,0.005498,0.006003,0.007058,0.007122,0.006147,0.005190,0.003969,0.002198
1596,6.556920e-14,9.937586e-12,8.203052e-10,3.687938e-08,9.030364e-07,1.204318e-05,8.747801e-05,3.461305e-04,7.469501e-04,8.882135e-04,...,0.004327,0.004108,0.004493,0.005383,0.005383,0.004650,0.004338,0.004737,0.004748,0.003321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2.532091e-14,3.962464e-12,3.368294e-10,1.555395e-08,3.901950e-07,5.318075e-06,3.938008e-05,1.584390e-04,3.463544e-04,4.113941e-04,...,0.002230,0.005722,0.008738,0.007930,0.005451,0.004239,0.003725,0.003521,0.003113,0.001850
1216,6.498664e-37,3.164092e-33,9.405860e-30,1.707153e-26,1.891781e-23,1.279953e-20,5.287391e-18,1.333563e-15,2.053572e-13,1.930772e-11,...,0.001557,0.001361,0.002160,0.004062,0.006446,0.008184,0.008239,0.006350,0.003632,0.001568
1653,2.681781e-124,1.962229e-116,7.860433e-109,1.723907e-101,2.069910e-94,1.360692e-87,4.897093e-81,9.649116e-75,1.040895e-68,6.147481e-63,...,0.007572,0.010007,0.009521,0.006121,0.003086,0.003215,0.005001,0.004906,0.002661,0.000791
559,1.939574e-38,1.457704e-34,6.500703e-31,1.720203e-27,2.701019e-24,2.516540e-21,1.391261e-18,4.563957e-16,8.883884e-14,1.026107e-11,...,0.000917,0.001140,0.002325,0.004521,0.006503,0.006881,0.005738,0.004025,0.002411,0.001159


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X

array([[1.55580913e-026, 1.60749287e-024, 1.63081422e-022, ...,
        1.94559249e-001, 9.20408901e-002, 4.22516988e-002],
       [2.87380757e-003, 9.64049700e-003, 2.76020425e-002, ...,
        1.13007938e-001, 1.13816193e-001, 9.00664962e-002],
       [2.98060427e-103, 8.37678012e-098, 2.20794491e-092, ...,
        2.78500504e-001, 3.05064975e-001, 2.64921920e-001],
       ...,
       [4.82084081e-114, 7.34369763e-108, 9.88899775e-102, ...,
        2.28611268e-001, 1.36119013e-001, 5.92493119e-002],
       [3.48663030e-028, 5.45549678e-026, 8.17835786e-024, ...,
        1.86381636e-001, 1.23256671e-001, 8.68272224e-002],
       [1.09869844e-029, 2.04738805e-027, 3.64302111e-025, ...,
        2.22054842e-001, 1.68018399e-001, 1.53311731e-001]])

In [26]:
# KNN Classifier
def knn_classifier(train_x, train_y):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(train_x, train_y)
    return model

def CNN(train_x, train_y, exp_max=1350,exp_min=0):
    #CNN hyperparameters
    BATCH_SIZE=128
    n_input = exp_max - exp_min
    n_classes = 5 
    
    
    # Define network structure
    model = Sequential()

    model.add(K.layers.Conv1D(32, 8,strides=8, padding='same',input_shape=(n_input,1), activation='relu'))
    model.add(K.layers.Conv1D(32, 5,strides=5, padding='same', activation='relu'))
    model.add(K.layers.Conv1D(16, 3,strides=3, padding='same', activation='relu'))
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(1024,activation='relu'))
    model.add(K.layers.Dense(n_classes, activation='softmax'))
    #Define optimizer        
    optimizer = K.optimizers.rmsprop()

    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['categorical_accuracy'])
    
    train_x = train_x.reshape(train_x.shape[0],n_input,1)
    model.fit(train_x,train_y,batch_size=BATCH_SIZE, nb_epoch=15,
                         verbose=1)
    
    return model

In [22]:
#convert the labels to onehot encoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_true)
encoded_Y = encoder.transform(y_true)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [23]:
n_fold = 5
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=n_fold, shuffle=True,random_state=30)

In [27]:
test_classifier = ['KNN']
classifiers = {'KNN' :knn_classifier,
               'CNN': CNN
              }



In [29]:
accuracy_exp = np.empty((n_fold,1)) 
start_time = time.time()   
for classifier in test_classifier: 
    print ('******************* %s ********************' % test_classifier)
    for k, (train, test) in enumerate(k_fold.split(X, dummy_y)):       
           
                                            
    
            #predict experimental prediction accuracy
            if classifier == 'CNN':
                train_y = enc.transform(train_y.reshape(-1,1))
                model = classifiers[classifier](train_combine, train_y) 
                predict_exp = model.predict(np.expand_dims(X_exp[test],2))
                predict_exp = enc.inverse_transform(predict_exp)
            else:
                model = classifiers[classifier](train_combine, train_y) 
                predict_exp = model.predict(X_exp[test])
            
            accuracy_exp[k] = accuracy_score(y_exp[test], predict_exp)  
            print ('accuracy_exp: %.2f%%' % (100 * accuracy_exp[k]))
            # 5 fold cross validation

******************* ['KNN'] ********************


NameError: name 'train_combine' is not defined